### Prelude:

In [1]:
import numpy as np
from matplotlib import pyplot as plt
import pandas as pd
import seaborn as sns
import random
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import KFold
import statsmodels.api as sm
set_seed = 42

In [2]:
data = pd.read_csv("./../Datasets/cleaned_data.csv")

In [3]:
#from Tiffanie's code
# distribution of subjects across treatment conditions (like Table 1 from paper)
# N = 186
treatment_freq = data[["treatment_value", "treatment_frame"]].value_counts()
treatment_rel_freq = data["treatment_frame"].value_counts(normalize=True)
treatment_freq.to_frame().sort_index().join(treatment_rel_freq)

,,count,proportion
treatment_value,treatment_frame,,
0,No framing,22,0.118280
1,Positive science,22,0.118280
2,Negative science,18,0.096774
3,Religious,35,0.188172
4,Equity,30,0.161290
5,Efficiency,32,0.172043
6,Secular,27,0.145161


### Difference of Means Tables:

In [4]:
pd.pivot_table(data, values=['mean_climate_support_before', 'mean_climate_support_after', 'mean_climate_support_change'],
               index=['treatment_value','treatment_frame'],
               aggfunc=['mean'])

mean  \
                                 mean_climate_support_after   
treatment_value treatment_frame                               
0               No framing                         1.590909   
1               Positive science                   1.795455   
2               Negative science                   1.972222   
3               Religious                          1.857143   
4               Equity                             1.916667   
5               Efficiency                         1.953125   
6               Secular                            1.990741   

                                                              \
                                 mean_climate_support_before   
treatment_value treatment_frame                                
0               No framing                          1.545455   
1               Positive science                    1.670455   
2               Negative science                    1.763889   
3               Religious                           1.750000   
4               Equity                              1.866667   
5               Efficiency                          1.914062   
6               Secular                             1.879630   

                                                              
                                 mean_climate_support_change  
treatment_value treatment_frame                               
0               No framing                          0.045455  
1               Positive science                    0.125000  
2               Negative science                    0.208333  
3               Religious                           0.107143  
4               Equity                              0.050000  
5               Efficiency                          0.039062  
6               Secular                             0.111111

In [5]:
pd.pivot_table(data, values=["mean_climate_support_after", 'mean_climate_support_before', 'mean_climate_support_change'],
               index=["party_id", "treatment_frame"], aggfunc=['mean'])

mean  \
                               mean_climate_support_after   
party_id      treatment_frame                               
Democrat      Efficiency                         2.333333   
              Equity                             1.946429   
              Negative science                   2.550000   
              No framing                         1.725000   
              Positive science                   2.343750   
              Religious                          2.026316   
              Secular                            1.980769   
Independent   Efficiency                         1.946429   
              Equity                             1.900000   
              Negative science                   2.062500   
              No framing                         1.725000   
              Positive science                   1.468750   
              Religious                          1.694444   
              Secular                            2.000000   
No preference Efficiency                         1.750000   
              Equity                             2.333333   
              No framing                         0.500000   
              Positive science                   0.666667   
              Religious                          1.916667   
              Secular                            2.500000   
Republican    Efficiency                         1.100000   
              Equity                             1.718750   
              Negative science                   0.437500   
              No framing                         0.000000   
              Positive science                   2.333333   
              Religious                          1.375000   
              Secular                            1.666667   

                                                            \
                               mean_climate_support_before   
party_id      treatment_frame                                
Democrat      Efficiency                          2.270833   
              Equity                              1.946429   
              Negative science                    2.200000   
              No framing                          1.600000   
              Positive science                    2.187500   
              Religious                           1.947368   
              Secular                             1.865385   
Independent   Efficiency                          1.803571   
              Equity                              1.900000   
              Negative science                    2.000000   
              No framing                          1.700000   
              Positive science                    1.375000   
              Religious                           1.666667   
              Secular                             1.916667   
No preference Efficiency                          1.750000   
              Equity                              2.250000   
              No framing                          1.000000   
              Positive science                    0.666667   
              Religious                           1.750000   
              Secular                             2.500000   
Republican    Efficiency                          1.400000   
              Equity                              1.562500   
              Negative science                    0.437500   
              No framing                          0.000000   
              Positive science                    2.083333   
              Religious                           1.000000   
              Secular                             1.416667   

                                                            
                               mean_climate_support_change  
party_id      treatment_frame                               
Democrat      Efficiency                          0.062500  
              Equity                              0.000000  
              Negative science                    0.3500

### Linear Regression:

In [6]:
def lin_estimator_mult_treat_formula(data, y_var, treatment_vars, covariates):
    """
    Inputs:
        data: pandas dataframe containing all x and y columns
        y_var: name of y variable
        treatment_vars: 
        covariates: list of string names of covariate

    Returns: Lin estimator model, formula
    """
    # Demean the covariates
    for cov in covariates:
        data[cov + '_demeaned'] = data[cov].dropna() - data[cov].dropna().mean()

    # Define the regression formula
    # Include each treatment indicator
    treatments_formula = " + ".join(treatment_vars)

    # Include each interaction term (automatically includes individual covariates)
    interactions = []
    for treatment in treatment_vars:
        for cov in covariates:
            interactions.append(f"{cov+ '_demeaned'} * {treatment}")
    
    interactions_formula = " + ".join(interactions)

    # Full formula -- include any other control(s)
    formula = f"{y_var} ~ {treatments_formula} + {interactions_formula}"

    # Fit the regression model and save results object
    model = sm.OLS.from_formula(formula, data=data).fit()

    # Return results object with robust covariance type
    return model.get_robustcov_results(cov_type="HC3"), formula

def lin_estimator_mult_treat(data, y_var, treatment_vars, covariates):
    """
    Inputs:
        data: pandas dataframe containing all x and y columns
        y_var: name of y variable
        treatment_vars: 
        covariates: list of string names of covariate

    Returns: Lin estimator model NO FORMULA
    """
    # Demean the covariates
    for cov in covariates:
        data[cov + '_demeaned'] = data[cov].dropna() - data[cov].dropna().mean()

    # Define the regression formula
    # Include each treatment indicator
    treatments_formula = " + ".join(treatment_vars)

    # Include each interaction term (automatically includes individual covariates)
    interactions = []
    for treatment in treatment_vars:
        for cov in covariates:
            interactions.append(f"{cov+ '_demeaned'} * {treatment}")
    
    interactions_formula = " + ".join(interactions)

    # Full formula -- include any other control(s)
    formula = f"{y_var} ~ {treatments_formula} + {interactions_formula}"

    # Fit the regression model and save results object
    model = sm.OLS.from_formula(formula, data=data).fit()

    # Return results object with robust covariance type
    return model.get_robustcov_results(cov_type="HC3")

In [7]:
# method 2: "party" -- group by Democrat/Republican-leaning, then include or exclude pure Independents/no preference
data.loc[(data["party_id"] == "Democrat") | (data["QID74"] == 2), "party"] = "D"
data.loc[(data["party_id"] == "Republican") | (data["QID74"] == 4), "party"] = "R"
data.loc[(data["QID74"] == 3), "party"] = "I"

# party ID
party_id = data.loc[:, "party_id.1"]
data["party_cov"] = party_id.fillna(data["party_id.2"]).fillna(data["QID74"])
data.party_cov = data.party_cov.astype(int)

In [8]:
# unsure if I am encoding variables for treatment conditions correctly
# for now creating indicator variable per treatment condition
treat_data = pd.get_dummies(data, columns=["treatment_value"])
treat_data["pre_avg_policy_support"] = treat_data[['GasTax', 'CarbTax', 'Treaty',
                                                   'RegCarb']].mean(axis=1)



In [9]:
#NOT SURE IF THESE ARE RIGHT (ESPECIALLY THE TREATMENT VALUES) - NEED TO CHECK
# IF YOU EDIT THESE IT WILL EDIT INPUT FOR ALL REGRESSIONS
outcome_var = 'mean_climate_support_after'
covariates = ['Religiosity', 'Economic_Reasoning', 'ScientificConfidence', 'party_cov', 'pre_avg_policy_support']
treatment_vars = [f"treatment_value_{i}" for i in range(1, 7)]
control_var = 'mean_climate_support_before'

In [10]:
# first pass at using Lin estimator for regression
model1_results, model1_formula = lin_estimator_mult_treat_formula(treat_data,
                                                          "mean_climate_support_after",
                                                          treatment_vars,
                                                          covariates)
model1_results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                                OLS Regression Results                                
======================================================================================
Dep. Variable:     mean_climate_support_after   R-squared:                       0.885
Model:                                    OLS   Adj. R-squared:                  0.852
Method:                         Least Squares   F-statistic:                     50.47
Date:                        Fri, 26 Apr 2024   Prob (F-statistic):           8.54e-67
Time:                                13:32:21   Log-Likelihood:                -9.2106
No. Observations:                         185   AIC:                             102.4
Df Residuals:                             143   BIC:                             237.7
Df Model:                                  41                                         
Covariance Type:                          HC3                                         
=============================================================================================================================
                                                                coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------------------------------------------
Intercept                                                     1.7840      0.406      4.390      0.000       0.981       2.587
treatment_value_1[T.True]                                     0.1177      0.410      0.287      0.774      -0.692       0.928
treatment_value_2[T.True]                                     0.2162      0.423      0.511      0.610      -0.620       1.053
treatment_value_3[T.True]                                     0.1228      0.412      0.298      0.766      -0.692       0.938
treatment_value_4[T.True]                                     0.0434      0.409      0.106      0.916      -0.766       0.852
treatment_value_5[T.True]                                     0.0423      0.413      0.102      0.919      -0.773       0.858
treatment_value_6[T.True]                                     0.0431      0.444      0.097      0.923      -0.834       0.921
Religiosity_demeaned                                          0.1098      0.041      2.668      0.009       0.028       0.191
Religiosity_demeaned:treatment_value_1[T.True]               -0.1898      0.082     -2.324      0.022      -0.351      -0.028
Religiosity_demeaned:treatment_value_2[T.True]                0.0201      0.121      0.166      0.868      -0.219       0.260
Religiosity_demeaned:treatment_value_3[T.True]               -0.0664      0.056     -1.187      0.237      -0.177       0.044
Religiosity_demeaned:treatment_value_4[T.True]               -0.1907      0.084     -2.269      0.025      -0.357      -0.025
Religiosity_demeaned:treatment_value_5[T.True]               -0.1039      0.066     -1.586      0.115      -0.233       0.026
Religiosity_demeaned:treatment_value_6[T.True]               -0.2003      0.213     -0.940      0.349      -0.622       0.221
Economic_Reasoning_demeaned                                   0.1238      0.059      2.115      0.036       0.008       0.239
Economic_Reasoning_demeaned:treatment_value_1[T.True]        -0.1370      0.091     -1.504      0.135      -0.317       0.043
Economic_Reasoning_demeaned:treatment_value_2[T.True]        -0.2792      0.235     -1.190      0.236      -0.743       0.185
Economic_Reasoning_demeaned:treatment_value_3[T.True]        -0.0877      0.158     -0.555      0.580      -0.400       0.225
Economic_Reasoning_demeaned:treatment_value_4[T.True]        -0.1400      0.081     -1.729      0.086      -0.300       0.020
Economic_Reasoning_demeaned:treatment_value_5[T.True]        -0.1340      0.193     -0.695      0.488      -0.515       0.247
Economic_Reasoning_demeaned:treatment_value_6[T.True]         0.0173      0.302      0.057      0

### Cross-Validation Regression:

In [11]:
#Randomly split data into two folds
shuffled = treat_data.sample(frac=1, random_state=set_seed)
split = np.array_split(shuffled, 2)
fold1 = split[0]
fold2 = split[1]

/opt/anaconda3/lib/python3.11/site-packages/numpy/core/fromnumeric.py:59: FutureWarning: 'DataFrame.swapaxes' is deprecated and will be removed in a future version. Please use 'DataFrame.transpose' instead.
  return bound(*args, **kwds)


In [12]:
# Function to extract treatment effects from model
def extract_treatment_effects(model, treatment_vars):
    coefs = dict(zip(model.model.exog_names, model.params))
    effects = {var: coefs.get(var, 0) if coefs.get(var) is not None else 0 for var in treatment_vars}
    return effects

# Function to find the best treatment
def find_best_treatment(effects):
    if effects:
        return max(effects, key=effects.get)
    return None

# Function to assign the best treatment indicator
def assign_best_treatment_indicator(data, best_treatment):
    if best_treatment:
        data['best_treatment_indicator'] = (data[best_treatment] == 1).astype(int)
    else:
        data['best_treatment_indicator'] = 0

# Function to estimate the effect of the best treatment condition
def estimate_best_treatment_effect(data, outcome, indicator, covariates):
    formula = f"{outcome} ~ {indicator} + {' + '.join(covariates)}"
    model = sm.OLS.from_formula(formula, data=data).fit()
    return model.summary()


In [13]:
# Example usage
effects_fold1 = extract_treatment_effects(lin_estimator_mult_treat(fold1, outcome_var, treatment_vars, covariates), treatment_vars)
effects_fold2 = extract_treatment_effects(lin_estimator_mult_treat(fold2, outcome_var, treatment_vars, covariates), treatment_vars)

best_treatment_fold1 = find_best_treatment(effects_fold1)
best_treatment_fold2 = find_best_treatment(effects_fold2)

assign_best_treatment_indicator(fold1, best_treatment_fold2)
assign_best_treatment_indicator(fold2, best_treatment_fold1)

combined_data = pd.concat([fold1, fold2])
final_result = estimate_best_treatment_effect(combined_data, outcome_var, 'best_treatment_indicator', covariates)

print(final_result)


                                OLS Regression Results                                
Dep. Variable:     mean_climate_support_after   R-squared:                       0.843
Model:                                    OLS   Adj. R-squared:                  0.838
Method:                         Least Squares   F-statistic:                     159.6
Date:                        Fri, 26 Apr 2024   Prob (F-statistic):           6.88e-69
Time:                                13:32:21   Log-Likelihood:                -38.045
No. Observations:                         185   AIC:                             90.09
Df Residuals:                             178   BIC:                             112.6
Df Model:                                   6                                         
Covariance Type:                    nonrobust                                         
                               coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------

/opt/anaconda3/lib/python3.11/site-packages/statsmodels/regression/linear_model.py:2013: RuntimeWarning: divide by zero encountered in divide
  self.het_scale = (self.wresid / (1 - h))**2


### Random Forest:

In [14]:
features = covariates + treatment_vars
folds = [fold1, fold2]

# Train models and predict outcomes
def train_and_predict(train_data, test_data):
    predictions = {}

    # Train a model for each treatment condition and predict for the test data
    for treatment in treatment_vars:
        # Assuming binary treatment, filter data where treatment is active
        treated_data = train_data[train_data[treatment] == 1]
        rf = RandomForestRegressor(n_estimators=100, random_state=42)
        rf.fit(treated_data[covariates], treated_data[outcome_var])
        # Store predictions for each treatment
        predictions[treatment] = rf.predict(test_data[covariates])

    return predictions

# Analyze both folds
results_fold1 = train_and_predict(fold2, fold1)  # Train on fold2, predict for fold1
results_fold2 = train_and_predict(fold1, fold2)  # Train on fold1, predict for fold2

def assign_best_condition(test_data, results):
    # Create a DataFrame from the results with appropriate indexing
    results_df = pd.DataFrame(results, index=test_data.index)
    # Use np.argmax on axis=1 to find the indices of maximum values along the horizontal axis
    best_treatment_indices = np.argmax(results_df.values, axis=1)
    # Convert indices to a Series to use the map function
    best_treatment_series = pd.Series(best_treatment_indices, index=test_data.index)
    # Map indices to treatment variable names
    best_treatment = best_treatment_series.map(dict(enumerate(treatment_vars)))
    test_data['best_personalized_treatment'] = best_treatment
    test_data['best_treatment_indicator'] = 1  # This assumes the personalized treatment is applied


# Assign best treatment based on opposite fold predictions
assign_best_condition(fold1, results_fold2)
assign_best_condition(fold2, results_fold1)

# Estimate the effect of the best personalized condition
def estimate_personalized_effect(data, treatment_indicator, outcome, covariates):
    formula = f"{outcome} ~ {treatment_indicator} + {' + '.join(covariates)}"
    model = sm.OLS.from_formula(formula, data=data).fit()
    return model.summary()

# Combining data for final analysis
combined_data = pd.concat([fold1, fold2])
effect_summary = estimate_personalized_effect(combined_data, 'best_treatment_indicator', outcome_var, covariates)
print(effect_summary)

                                OLS Regression Results                                
Dep. Variable:     mean_climate_support_after   R-squared:                       0.843
Model:                                    OLS   Adj. R-squared:                  0.839
Method:                         Least Squares   F-statistic:                     192.2
Date:                        Fri, 26 Apr 2024   Prob (F-statistic):           5.42e-70
Time:                                13:32:22   Log-Likelihood:                -38.192
No. Observations:                         185   AIC:                             88.38
Df Residuals:                             179   BIC:                             107.7
Df Model:                                   5                                         
Covariance Type:                    nonrobust                                         
                               coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------